In [8]:
import pandas as pd
from itertools import combinations
import os
from bidict import bidict
import pickle
import lfp_analysis.LFP_recording as LFP_recording
import importlib
importlib.reload(LFP_recording)

def hex_2_rgb(hex_color): # Orange color
    rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5))
    return rgb_color

def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):   
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, 'rb') as file:
        return(pickle.load(file))

In [10]:
df = pd.read_excel(r"C:\Users\megha\Documents\Padilla-Coreano\homecage_social_ephys_vids\pilot2\channel_mapping_sme.xlsx")
spike_cols = [col for col in df.columns if 'spike_interface_' in col.lower()]

# Extract brain regions from column names
# Assumes format 'spike_interface_REGION'
brain_regions = [col.split('spike_interface_')[1] for col in spike_cols]

# Create nested dictionary
subject_to_channel_dict = {}

for _, row in df.iterrows():
    subject = row['Subject'].astype(str)
    # Initialize inner dictionary for this subject
    subject_to_channel_dict[subject] = {}
    
    # Populate inner dictionary with brain region: spike value pairs
    for col, region in zip(spike_cols, brain_regions):
        subject_to_channel_dict[subject][region] = int(row[col])
print(subject_to_channel_dict)

{'1.1': {'mPFC': 19, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '1.2': {'mPFC': 26, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '1.3': {'mPFC': 9, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '2.1': {'mPFC': 15, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '2.2': {'mPFC': 7, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '2.3': {'mPFC': 19, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '2.4': {'mPFC': 10, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '3.1': {'mPFC': 25, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '3.2': {'mPFC': 26, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '3.3': {'mPFC': 26, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '4.1': {'mPFC': 25, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}, '4.4': {'mPFC': 25, 'vHPC': 31, 'BLA': 30, 'NAc': 28, 'MD': 29}}


In [11]:
data_path = r'D:\cups\data'
behavior_dicts = unpickle_this("cups_behavior_dicts.pkl")
print(behavior_dicts.keys())
recording_to_subject = {}
recording_to_behavior_dict = {}
for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith('merged.rec'):
            subject = str(int((file.split("_")[0]))/10)
            
            if subject in behavior_dicts.keys():
                recording_to_subject[file] = subject
                behavior_dict = behavior_dicts[subject]
                recording_to_behavior_dict[file] = behavior_dict



            

dict_keys(['1.1', '1.3', '2.1', '2.2', '2.3', '2.4', '3.1', '3.2', '4.1', '4.4'])


In [ ]:
merged_rec_path = r"D:\cups\data\11_cups_p4.rec\11_cups_p4_merged.rec"
rec_test = LFP_recording.LFPRecording(subject = '1.1', 
                                      behavior_dict=behavior_dicts['1.1'], 
                                      channel_dict=subject_to_channel_dict['1.1'], 
                                      merged_rec_path=merged_rec_path)


c:\Users\megha\Documents\GitHub\diff_fam_social_memory_ephys\lfp\trodes\read_exported.py:168: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


In [ ]:

rec_test.plot_to_find_threshold(threshold = 0)

TypeError: zscore() got an unexpected keyword argument 'scaling'